### 【問題1】機械翻訳の実行とコードリーディング

CodeReadingは都度markdownで書いています

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!wget http://www.manythings.org/anki/fra-eng.zip

--2019-03-30 02:39:46--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:30::6818:6dc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3305307 (3.2M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   3.15M  11.6MB/s    in 0.3s    

2019-03-30 02:39:46 (11.6 MB/s) - ‘fra-eng.zip’ saved [3305307/3305307]



In [0]:
!unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [0]:
#import する
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [0]:
# hyperparameterを設定する
batch_size = 64  # Batch size for training.
epochs = 25  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

単語をリストに入れていく

In [0]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [0]:
# 並び替え
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [0]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


単語をIDに変換する辞書を作る

zero配列の作成

In [0]:
# 単語をIDに変換する辞書を作る
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

onehot_encodingする

In [0]:
# onehot_encodingする
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


train用のencoderを作成する

In [0]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


train用のdecoderを作成する

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

modelを作成し学習させる1

In [0]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/25
8000/8000 [==============================] - 20s 3ms/step - loss: 0.9212 - val_loss: 0.9626
Epoch 2/25
8000/8000 [==============================] - 18s 2ms/step - loss: 0.7330 - val_loss: 0.7784
Epoch 3/25
8000/8000 [==============================] - 17s 2ms/step - loss: 0.6199 - val_loss: 0.6888
Epoch 4/25
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5634 - val_loss: 0.6543
Epoch 5/25
8000/8000 [==============================] - 18s 2ms/step - loss: 0.5284 - val_loss: 0.6148
Epoch 6/25
8000/8000 [==============================] - 17s 2ms/step - loss: 0.4960 - val_loss: 0.5788
Epoch 7/25
8000/8000 [==============================] - 18s 2ms/step - loss: 0.4691 - val_loss: 0.5544
Epoch 8/25
8000/8000 [==============================] - 18s 2ms/step - loss: 0.4469 - val_loss: 0.5355
Ep

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


predict用のmodelを作成する

数時から文字に変換する

In [0]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

予測用のmodleを使用する関数を作る

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

予測するためのループを回す

In [0]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Tu le chance !

-
Input sentence: Hi.
Decoded sentence: Attrapez !

-
Input sentence: Run!
Decoded sentence: Arrête de te la sécurée !

-
Input sentence: Run!
Decoded sentence: Arrête de te la sécurée !

-
Input sentence: Wow!
Decoded sentence: Quel chautte !

-
Input sentence: Fire!
Decoded sentence: Arrête de te la sécure !

-
Input sentence: Help!
Decoded sentence: Bon !

-
Input sentence: Jump.
Decoded sentence: Regardez ça.

-
Input sentence: Stop!
Decoded sentence: Arrête de te la sécurée !

-
Input sentence: Stop!
Decoded sentence: Arrête de te la sécurée !

-
Input sentence: Stop!
Decoded sentence: Arrête de te la sécurée !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Go on.
Decoded sentence: Prends les miennes.

-
Input sentence: Go on.
Decoded sentence: Prends les miennes.

-
Input sentence: Go on.
Decoded sentence: Prends les miennes.

-
Input sentence: Hel

【問題2】イメージキャプショニングの学習済みモデルの実行

別のnotebook参照してください

【問題3】Kerasで動かしたい場合はどうするかを調査

キャプション用のモデルをKerasで作成する

PyTorchで訓練されたウェイトの形式をKeras用の形式に変更する

【問題4】（アドバンス課題）コードリーディングと書き換え

In [6]:
import torch
import torch.nn as nn
import torchvision.models as models
#Alexnet
#VGG
#ResNet
#SqueezeNet
#DenseNet
#Inception v3
#GoogLeNet などがmodelsにある
from torch.nn.utils.rnn import pack_padded_sequence # シーケンス方向のpaddingで文章の長さを揃える

In [7]:
class EncorderCNN(nn.Module):
    def __init__(self, embed_size):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(EncorderCNN, self).__init__()#継承
        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-1]# スライスを用いて任意の層を取り出す[-1]なので最後(FC)以外を使用
        self.resnet = nn.Sequential(*models)# taple割り
        self.linear = nn.Linear(resnet.fc.in_features, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)
    
    def forward(self, images):
        """Extract feature vectors from input images."""
        with torch.no_grad(): # 計算グラフを作らない方法 backpropaしない #resnetの重みは凍結
            features = self.resnet(images)
        features = features.reshape(features.size(0),-1)
        features = self.bn(self.linear(features))
        return features
    
    
class DecorderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, max_seq_length=20):
        """Set the hyper-parameters and build the layers."""
        super(DecorderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.max_seg_length = max_seq_length
        
    def forward(self, features, captions, lengths):
        """Decode image features vectors and generates captions."""
        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1) #cat concatnate unsqueezeはaxis1に次元を追加
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True)
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs
    
    def sample(self, features, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []
        inputs = features.unsqueeze(1)
        for i in range(self.max_seg_length):
            hiddens, states = self.lstm(inputs, states)
            outputs = self.linear(hiddens.squeeze(1))
            _, predicted = outputs.max(1)
            sampled_ids.append(predicted)
            inputs = self.embed(predicted)
            inputs = inputs.unsqueeze(1)
        sample_ids = torch.stack(sampled_ids, 1)
        return smapled_ids

In [8]:
# kerasでmodel.py

from keras.layers import Dense, Reshape, Embedding, Concatenate, LSTM, Input, Activation
from keras.models import Model
from keras.applications.resnet50 import ResNet50


Using TensorFlow backend.


In [9]:
embed_size = 10
captions = 20
hidden_size = 30


input_tensor = Input(shape=(224,224,3))
input_word = Input(shape=(None,))
res = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3),pooling='avg')#GlobalAveragePooling
resnet = res(input_tensor)
features = Dense(embed_size)(resnet)#出力の形を整える
features = Reshape((1,embed_size))(features)# sequenceを追加

embedding = Embedding(captions, embed_size)(input_word)#正解ラベルの分散表現化
embedding = Concatenate(1)([features, embedding])#imageのfeatureとwordをsequence方向に結合 二値入力はリストで
lstm = LSTM(hidden_size, return_sequences=True)(embedding)
output = Dense(captions)(lstm)
output = Activation('softmax')(output)

model = Model([input_tensor,input_word],output)

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 2048)         23587712    input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10)           20490       resnet50[1][0]                   
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
reshape_1 